In [4]:
import tensorflow as tf
import numpy as np

In [2]:
#list out our bandits
#bandit 4 (index 3) is set to provide
#most often
bandits = [.2,0,-.2,-5]
num_bandits = len(bandits)

def pull_bandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [9]:
#init our agent
tf.reset_default_graph()

#Establish the feedfoward part of the network
#this does the actual choosing

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

#the enxt six lines establish the training procedure
#We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network
reward_holder = tf.placeholder(shape=[1], dtype = tf.float32)
action_holder = tf.placeholder(shape=[1], dtype = tf.int32)
responsible_weight = tf.slice(weights, action_holder,[1])
loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = .001)
update = optimizer.minimize(loss)

In [10]:
#train the agent

total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1 #The chance of taking a random action

In [16]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)

init = tf.initialize_all_variables()

#launch TF graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        #choose either random action or one from our network
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        #get our reward from picking one of the bandits
        reward = pull_bandit(bandits[action])
                #update the network
        _,resp,ww = sess.run([update, responsible_weight, weights],
                             feed_dict={reward_holder:[reward],
                                        action_holder:[action]})
            
            #ypdate our running tally of scores
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print("The agent thinks bandit " + str(np.argmax(ww)+1 + " is the most promising..."))
if np.argmax(ww) == np.argmax(-np.array(bandiuts)):
    print("... and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [1. 0. 0. 0.]
Running reward for the 4 bandits: [-2. -2.  0. 39.]
Running reward for the 4 bandits: [-4. -2.  1. 84.]
Running reward for the 4 bandits: [ -4.   0.  -1. 130.]
Running reward for the 4 bandits: [ -4.   0.   0. 179.]
Running reward for the 4 bandits: [ -5.   2.   0. 226.]
Running reward for the 4 bandits: [ -6.   2.   0. 275.]
Running reward for the 4 bandits: [ -7.   2.   0. 324.]
Running reward for the 4 bandits: [ -9.   3.   0. 369.]
Running reward for the 4 bandits: [-11.   1.   1. 412.]
Running reward for the 4 bandits: [-10.   0.   0. 459.]
Running reward for the 4 bandits: [-11.   2.   1. 505.]
Running reward for the 4 bandits: [-11.   2.   1. 555.]
Running reward for the 4 bandits: [-12.   2.   0. 603.]
Running reward for the 4 bandits: [-14.   2.  -1. 648.]
Running reward for the 4 bandits: [-14.   2.  -2. 695.]
Running reward for the 4 bandits: [-14.   1.  -3. 743.]
Running reward for the 4 bandits: [-13.   1.  -3. 788.]
Running 

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U25') dtype('<U25') dtype('<U25')